# Interpretting MuJoCo using the Gym environment

OpenAI has created a Gym environment for interacting with MuJoCo. This notebook will give you the basics of how to interact with the environment and how to interpret it

### Import libraries

Import the `gym` environment to interact with environment.
We also will use `numpy` to make interacting with the data more convenient

In [1]:
import gym
import numpy as np

### Load the MuJoCo environment

Load a MuJoCo environment. We will use the 'Ant-v3' environment which has the goal of maximizing movement to the right.

To better understand the environment look at the [model used](https://gym.openai.com/envs/Ant-v2/). We will want to train the model to walk.
(Note, the model linked is called 'Ant-v2', this is because the 'Ant-v2' and 'Ant-v3' environments use the same character model)

In [2]:
env = gym.make('Ant-v3')
obs = env.reset()

### Observation and Action spaces

The agent's observation in this environment is a vector of length 111.

We will see how to interpret this observation soon

In [3]:
print(f"Total of {env.observation_space.shape[0]} observations")
print(obs)

Total of 111 observations
[ 7.66922597e-01  9.95416633e-01  4.60097523e-02  2.78333217e-02
  7.90830947e-02 -4.81468545e-02 -9.48028434e-02 -2.60754144e-02
  5.19646702e-02  3.69375576e-02 -9.00599800e-02 -9.66656109e-02
  8.01901452e-02 -1.70890899e-01 -2.25243027e-02 -7.70366981e-02
  4.04252178e-02 -8.72032049e-02  7.71853388e-02  7.42523956e-02
 -1.14537546e-02  8.59330443e-02  1.55663910e-02  5.76117157e-02
  7.04248884e-02 -7.93171507e-02 -7.84296295e-04  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00

The action space for this environment is of length 8 which cooresponds to the number of joints on the ant.
Actions are provided to the ant by giving a vector of length 8, telling the environment what angles to move the joints to.

In [4]:
print(f"Total of {env.action_space.shape[0]} actions")

Total of 8 actions


### Interpretting observations

Observations have three components: `qpos`, `qvel` and `cfrc_ext` which represent the following:
- `qpos`: Positions of parts of the ant
- `qvel`: Velocities of parts of the ant
- `cfrc_ext`: External contact forces on different parts of the ant. Each part has an associated 6D-vector that is in the form [torque_xyz, force_xyz]

Full documentation is available at this [link](https://mujoco.readthedocs.io/en/latest/APIreference.html?highlight=cfrc_ext#mjdata)

In [5]:
print(f"Num position observations = {env.sim.data.qpos.shape}")
print(f"Num velocity observations = {env.sim.data.qvel.shape}")
print(f"Num of external contact force vectors = {env.sim.data.cfrc_ext.shape}")
print(f"Num external contact force observations = {env.sim.data.cfrc_ext.flat.copy().shape}")
print(f"Total possible observations = {env.sim.data.qpos.shape[0] + env.sim.data.qvel.shape[0] + env.sim.data.cfrc_ext.flat.copy().shape[0]}")


Num position observations = (15,)
Num velocity observations = (14,)
Num of external contact force vectors = (14, 6)
Num external contact force observations = (84,)
Total possible observations = 113


Note that the total possible observations is 113 while `env.observation_space` has a size of 111. This is because the first 2 elements of `qpos` are the absolute (x, y) position of the ant. However, this absolute reference frame shouldn't be needed for the objective of moving right. Because of this, the `gym` environment drops these values by default.

In [6]:
# https://roboti.us/forum/index.php?threads/does-d-cfrc_ext-give-torque-force-or-opposite.3368/
# The 6D spatial vectors that start with "c" have their rotational component first, followed by the translational component. [torque, force] 
env.sim.data.cfrc_ext.shape


(14, 6)

In [7]:
env.get_body_com("torso")

array([-0.09916426, -0.03001602,  0.7669226 ])

In [8]:
env.state_vector().shape

(29,)